# FOD Dataset Cleaning Script
With this script FOD will be cleaning the Agency Dataset delivered by Informa. FOD will remap, recalculate and fill the dataset according to what is stated in the Data Quality Rapport.

## Importing the libraries

To get started with cleaning, FOD will need to use libraries. These Libraries are standard within the Anaconda distribution and can be imported without requiring a new file.

In [ ]:
import sklearn as sk
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)
print('scikit-learn version:', sk.__version__)
print('matplotlib version:', matplotlib.__version__)

%matplotlib inline

## Importing the dataset
FOD needs to load the data into Python to get started with the cleaning process.
This is done by setting variables for `datafolder` and `filename`. As Informa gave us the dataset over Canvas, FOD can not use an online download link and the dataset has to be downloaded seperately.

The dataset will be loaded into the variable `df` this is done in the following process:
```py
datafolder = 'data/'
filename = 'dataset_informa.csv'
df = pd.read_csv(datafolder + filename, sep=',', low_memory=False, encoding = 'ISO-8859-1')
```

To test if the import process has gone successfull FOD will look at the shape of the dataset and the `head()`.
The `head()` method shows the first 5 values within the dataset.
```py
print(df.shape)
df.head()
```

In [ ]:
datafolder = 'data/'
filename = 'dataset_informa.csv'
df = pd.read_csv(datafolder + filename, sep=',', low_memory=False, encoding = 'ISO-8859-1')
print(df.shape)
df.head()

## Removing unused columns
Currently FOD only planned to remove the `RETENTION_RATIO` column from this dataset. This is again stated in the Data Quality Rapport.
```py
df.drop(['RETENTION_RATIO'], axis = 1, inplace = True)
```
## Replacing missing values
FOD will start with `PRIMARY_AGENCY_ID`, there are 0 values that will be replaced with the original Agency Id.
To find these values FOD will use the `np.where()`, this method allows for filtering and replacement. If the value of it is 0, it needs to be replaced with it's original Id.
```py
df['PRIMARY_AGENCY_ID'] = np.where(df['PRIMARY_AGENCY_ID'] == 0, df['PRIMARY_AGENCY_ID'], df['AGENCY_ID'])
```

For the `VENDOR_END_YEAR` FOD want to replace the 0 values. This can either mean the agency never used a Vendor or has stopped using it.
To check this, FOD will look at the `VENDOR_START_YEAR`, if this start year is 0 it means it wasn't used, in that case FOD will set the `END_YEAR` value to 1 to indicate it was never used.

```py
df['CL_END_YEAR'] = np.where(df['CL_START_YEAR'] == 0, 0, 1)
df['PL_END_YEAR'] = np.where(df['PL_START_YEAR'] == 0, 0, 1)
```

Next up is `AGENCY_APPOINTMENT_YEAR`. This column has 5000 missing values that will need to be replaced. To do this FOD will first save the mean of the known data in the `mean` variable. After this is done, FOD will replace the `99999` values with the value in `mean`
```py
mean = df['AGENCY_APPOINTMENT_YEAR'].mean()
df['AGENCY_APPOINTMENT_YEAR'].replace(99999, mean, inplace=True)
```

To replace all other missing values, FOD will call the `replace()` method on the data frame to replace the `99999` values with `0`.
```py
df.replace(99999, 0, inplace=True)
```

In [ ]:
df.drop(['RETENTION_RATIO'], axis = 1, inplace = True)

# Primary_Agency_ID: When this column data is missing/99999 we are going to replace it with the Agency_ID. 
# We can not find any way to associate a primary agency with the agency in question so will replace it with its ID.
df['PRIMARY_AGENCY_ID'] = np.where(df['PRIMARY_AGENCY_ID'] == 0, df['PRIMARY_AGENCY_ID'], df['AGENCY_ID'])

# (VENDOR)_END _YEAR: The year the agency stopped using the vendor. 
# This can either mean the vendor is never used or the vendor is still in use. 
# If the vendor has never been used we can replace the value with 0, if the vendor is still in use we can replace it with 1.
df['CL_END_YEAR'] = np.where(df['CL_START_YEAR'] == 0, 0, 1)
df['PL_END_YEAR'] = np.where(df['PL_START_YEAR'] == 0, 0, 1)

# Agency_appointment_year: 5000 missing data, irrelevant column, will be replaced with average.
mean = df['AGENCY_APPOINTMENT_YEAR'].mean()
df['AGENCY_APPOINTMENT_YEAR'].replace(99999, mean, inplace=True) 

# replace the rest of the 99999 values with 0
df.replace(99999, 0, inplace=True)


## Recalculating Incomplete Data
There are incomplete months within the dataset. FOD wants to have a dataset that is filled with usefull data. FOD has decided to recalculate 4 of the key columns to get the months up to 12. First FOD need to seperate the incomplete months from the completed months. FOD will do that using indexing on the dataframe. Within this codeblock FOD also prepairs the `recal` list, this list is the columns that need to be recalculated.
```py
recal = ["POLY_INFORCE_QTY", "NB_WRTN_PREM_AMT", "WRTN_PREM_AMT", "PRD_ERND_PREM_AMT", "PRD_INCRD_LOSSES_AMT"]
dfincomplete = df[(df["MONTHS"] != 12)]
```
Once this new dataframe is gotten FOD wants to loop through each row. This can be done using the `iterrows()` within Pandas.
FOD will then set a temporary variable to save the months in to easily calculate with that value.
```py
for index, row in df.iterrows():
    #Amount of months
    months = row["MONTHS"]
```
FOD then wants to start calculating the new values. This is done by looping through the items in the `recal` list. The `newvalue` is the old value times 12 minus the amount of months present, this value will then be set in the main DataFrame.
```py
for column in recal:
    newvalue = row[column] * (12 - months) #Value times amount of months missing
    df.set_value(index, column, newvalue) #Set new row in dataset
```
Once all columns are recalculated FOD wants to put the `MONTHS` variable back to 12 so it indicates the row has been cleaned.
```py
df.set_value(index, "MONTHS", 12) #Reset the months to be 12
```

In [ ]:
#Get a list of all values needing to be recalculated per item
recal = ["POLY_INFORCE_QTY", "NB_WRTN_PREM_AMT", "WRTN_PREM_AMT", "PRD_ERND_PREM_AMT", "PRD_INCRD_LOSSES_AMT"]
dfincomplete = df[(df["MONTHS"] != 12)]
for index, row in df.iterrows():
    #Amount of months
    months = row["MONTHS"]
    for column in recal:
        newvalue = row[column] * (12 - months) #Value times amount of months missing
        df.set_value(index, column, newvalue) #Set new row in dataset
    df.set_value(index, "MONTHS", 12) #Reset the months to be 12

## Recalculate missing data within RETENTION_RATIO and LOSS_RATIO.
Since there are new values in columns that are used in calculating, FOD will need to also recalculate the calculated columns. Within the documentation is said that `LOSS_RATIO` column is the `PRD_INCRD_LOSSES_AMT` devided by `WRTN_PREM_AMT`.
To start off FOD wants to iterrate over the rows within the dataset. To check if we can calculate the value, we will check if the `WRTN_PREM_AMT` and `PRD_INCRD_LOSSES_AMT` are greater than 0. If this is the case we put the new `LOSS_RATIO` value into the DataFrame.
```py
for index, row in df.iterrows():
    if (row["WRTN_PREM_AMT"] > 0) & (row["PRD_INCRD_LOSSES_AMT"] > 0):
        df.set_value(index, "LOSS_RATIO", row["PRD_INCRD_LOSSES_AMT"] / row["WRTN_PREM_AMT"])
```
If this is not the case, we will just put 0 into the DataFrame.
```py
else:
    df.set_value(index, "LOSS_RATIO", 0)
```
Because our calculations can go wrong FOD wants to filter out our new missing values.
```py
df['LOSS_RATIO'] = df['LOSS_RATIO'].replace(99998, 0)
df['LOSS_RATIO'] = df['LOSS_RATIO'].replace(99997, 0)
```

In [ ]:
#LOSS_RATIO = PRD_INCRD_LOSSES_AMT / WRTN_PREM_AMT

for index, row in df.iterrows():
    if (row["WRTN_PREM_AMT"] > 0) & (row["PRD_INCRD_LOSSES_AMT"] > 0):
        df.set_value(index, "LOSS_RATIO", row["PRD_INCRD_LOSSES_AMT"] / row["WRTN_PREM_AMT"])
    else:
        df.set_value(index, "LOSS_RATIO", 0)
    
df['LOSS_RATIO'] = df['LOSS_RATIO'].replace(99998, 0)
df['LOSS_RATIO'] = df['LOSS_RATIO'].replace(99997, 0)


# Filling missing data with the mean
Because FOD wants to make the dataset as complete as possible, FOD also wants to replace missing data with valuable data,
For this FOD wants to recalculate the previous columns stated in the `recall` variable. For the purpose of visualization FOD will also recalculate the `PREV_POLY_INFORCE_QTY` and `NB_WRTN_PREM_AMT`

To start off FOD wants to collect all products and product lines. This is done by calling the `unique()` method on a Series.
```py
states = df.STATE_ABBR.unique()
prod_line = df.PROD_LINE.unique()
```
We want to calculate the average of a state and a product into an dictonairy. This will be done only once so we can save it and re-use it.
```py
state_dict = {}    
for state in states:
    for line in prod_line:
        state_mean = df[(df["STATE_ABBR"] == state) & (df["PROD_LINE"] == line)].mean()
        state_dict[(state, line)] = state_mean
```

In [ ]:
average = ["PREV_POLY_INFORCE_QTY","POLY_INFORCE_QTY", "NB_WRTN_PREM_AMT", "WRTN_PREM_AMT", "PRD_ERND_PREM_AMT"]

states = df.STATE_ABBR.unique()
prod_line = df.PROD_LINE.unique()
   
state_dict = {}    
for state in states:
    for line in prod_line:
        state_mean = df[(df["STATE_ABBR"] == state) & (df["PROD_LINE"] == line)].mean()
        state_dict[(state, line)] = state_mean


## Filling the data
For this FOD will use the same trick as before. FOD starts out by iterating over the rows and checking if the value of one of the columns is 0. If this is the case we try filling the value with a precalculated average. If this goes wrong we just fill it with 0 as we can not generate values out of nothing.
```py
for index, row in df.iterrows():
    for col in average:
        if(row[col] == 0):
            try:
                newvalue = state_dict[(row["STATE_ABBR"], row["PROD_LINE"])]
            except(ValueError): #No average can be found since its all 0
                newvalue = 0 
            df.set_value(index, col, newvalue[col])
```

In [ ]:
for index, row in df.iterrows():
    for col in average:
        if(row[col] == 0):
            try:
                newvalue = state_dict[(row["STATE_ABBR"], row["PROD_LINE"])]
            except(ValueError): #No average can be found since its all 0
                newvalue = 0 
            df.set_value(index, col, newvalue[col])

## Mapping values to a number
Because FOD can not visualise words that easily, FOD is going to remap these values to numbers. FOD does this by first making dictonaries for every value that will be remapped.
```py
statedict = {np.NaN : 0, "IN" : 1, "KY" : 2, "MI" : 3, "OH" : 4, "PA" : 5, "WV" : 6}
proddict = {np.NaN : 0, "BOILERMACH" : 1, "BOP" : 2, "COMMAUTO" : 3, "COMMINLMAR" : 4, "COMMPOL": 5, "COMMUMBREL" : 6, "CRIME" : 7,
           "FIREALLIED" : 8, "GARAGE" : 9, "GENERALIAB" : 10, "WORKCOMP" : 11, "ANNIV" : 12, "ANNIV   12":13, "CYCLES":14,
           "DTALK":15, "DWELLFIRE":16, "HOMEONWERS":17,"MOBILEHOME":18,"MOTORHOM12":19,"MOTORHOME":20,"PERSUMBREL":21, "YACHT":22,
           "DTALK   12":23, "PERSINLMAR":24, "SNOWMOBILE":25,"CYCLES  12":26,"SNOWMOBBI12":27,"PERSAIP":28}
prodlinedict = {np.NaN : 0, "CL":1, "PL":2}
vendordict = {'Unknown': 0, 'A': 1, 'B':2, 'C' : 3, 'E':4}
```
After that FOD map every one of these to the columns to a new column with the `_ID` name. This is done using the `map()` function within Pandas.
```py
df['STATE_ID'] = df['STATE_ABBR'].map(statedict)
df['PROD_ID'] = df["PROD_ABBR"].map(proddict)
df['PROD_LINE_ID'] = df['PROD_LINE'].map(prodlinedict)
df['VENDOR_ID'] = df['VENDOR'].map(vendordict)
```

In [ ]:
statedict = {np.NaN : 0, "IN" : 1, "KY" : 2, "MI" : 3, "OH" : 4, "PA" : 5, "WV" : 6}
proddict = {np.NaN : 0, "BOILERMACH" : 1, "BOP" : 2, "COMMAUTO" : 3, "COMMINLMAR" : 4, "COMMPOL": 5, "COMMUMBREL" : 6, "CRIME" : 7,
           "FIREALLIED" : 8, "GARAGE" : 9, "GENERALIAB" : 10, "WORKCOMP" : 11, "ANNIV" : 12, "ANNIV   12":13, "CYCLES":14,
           "DTALK":15, "DWELLFIRE":16, "HOMEONWERS":17,"MOBILEHOME":18,"MOTORHOM12":19,"MOTORHOME":20,"PERSUMBREL":21, "YACHT":22,
           "DTALK   12":23, "PERSINLMAR":24, "SNOWMOBILE":25,"CYCLES  12":26,"SNOWMOBBI12":27,"PERSAIP":28}
prodlinedict = {np.NaN : 0, "CL":1, "PL":2}
vendordict = {'Unknown': 0, 'A': 1, 'B':2, 'C' : 3, 'E':4}

df['STATE_ID'] = df['STATE_ABBR'].map(statedict)
df['PROD_ID'] = df["PROD_ABBR"].map(proddict)
df['PROD_LINE_ID'] = df['PROD_LINE'].map(prodlinedict)
df['VENDOR_ID'] = df['VENDOR'].map(vendordict)

# Spliting and saving the dataset
To split the dataset into CL and PL FOD will need to first split it into 2 DataFrames. 
```py
df_cl = df[df.PROD_LINE=='CL']
df_pl = df[df.PROD_LINE=='PL']
```

FOD then wants to remove irrelevant columns from the split dataframes
```py
df_cl = df_cl.filter(regex='^(?!PL_)\w+', axis=1)
df_pl = df_pl.filter(regex='^(?!CL_)\w+', axis=1)
```
To get an end pro

In [ ]:
# split PL and CL datasets
df_cl = df[df.PROD_LINE=='CL']
df_pl = df[df.PROD_LINE=='PL']

# remove irrelevant columns from splits
df_cl = df_cl.filter(regex='^(?!PL_)\w+', axis=1)
df_pl = df_pl.filter(regex='^(?!CL_)\w+', axis=1)

# write dataframes to csv files
df_cl.to_csv('cleaned_CL_'+filename)
df_pl.to_csv('cleaned_PL_'+filename)